In [63]:
#import data from Victor
%store -r df
df['month_name'] = df.index.month_name()

#list_var_rawbox_check = ['battery_voltage', 'current_out', 'state_of_charge_percent', 'temperature', 'power_out']
option_resampling = [{'label':'month','value':'1MS'},{'label':'day','value':'1D'}]

In [64]:
df.head()

,battery_id,battery_voltage,current,current_in,current_out,panel_voltage,state_of_charge_percent,temperature,usb_current,voltage,...,power_in,net_power_battery,other_out_current,bool_current_out,bool_current_in,hour,minutes,day_n,hour:min,month_name
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-03-02 06:40:00,0,7.0750,-2.276,2.278,0.002,7.350,100.0,28.287938,0.002,7.280,...,16.116850,16.102700,0.000,0.0,1.0,6,40,1,6:40,March
2021-03-02 06:50:00,0,6.7200,0.000,0.002,0.002,0.005,102.0,27.152667,0.002,6.735,...,0.013440,0.000000,0.000,0.0,0.0,6,50,1,6:50,March
2021-03-02 07:00:00,0,6.7100,0.010,0.002,0.012,0.005,102.0,26.919203,0.002,6.725,...,0.013420,-0.067100,0.010,0.0,0.0,7,0,1,7:0,March
2021-03-02 07:10:00,0,6.7050,0.010,0.002,0.012,0.005,102.0,27.003891,0.002,6.720,...,0.013410,-0.067050,0.010,0.0,0.0,7,10,1,7:10,March
2021-03-02 07:20:00,0,6.7025,0.013,0.002,0.015,0.005,102.0,27.096590,0.002,6.715,...,0.013405,-0.087125,0.013,0.0,0.0,7,20,1,7:20,March


In [65]:
#import data from Elias
import pandas as pd
df_temp_features_clustered = pd.read_csv('df_temp_features_clustered.csv')
df_power_features_clustered = pd.read_csv('df_power_features_clustered.csv')

option_feature_type = [{'label':'Temperature features','value':'Temperature features'},
                        {'label':'Power features','value':'Power features'}]

In [ ]:
!pip install dash-bootstrap-components
!pip install jupyter-dash

In [28]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc

In [160]:
#Building the app
app = JupyterDash(__name__, external_stylesheets = [dbc.themes.SKETCHY],
                            meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}]) #if you want to make it adjust when the dashboard is visualized on mobile phones

In [161]:
#Building the layout
app.layout = dbc.Container([
    #------------------------------------------------------------------------------------------------------------------------
                    #Row #1: title row
                    dbc.Row([
                        html.H1('BBoxx Lithium-Ion Battery Field Data Challenge', className = 'text-center mb-4'),
                        html.P('Team 22: Nicole Schauser, Victor Bossard, Etienne Beauchamp, Elias Galiounas, Maria Varini', className = 'text-center mb-4')
                    ]), #close row 1
    #------------------------------------------------------------------------------------------------------------------------
                    #Row 2; sub-section title and text
                    dbc.Row([
                        dbc.Col([
                      html.H2('Time series analysis', className = 'text-center mb-4'),
                      html.P("A first look at the time series data provided.", className = 'text-center mb-4'),
                      dcc.Dropdown(id = 'var-dropdown',
                      value = 'voltage',
                       options = [{'label': a, 'value': a} for a in df.columns]),
                       html.Label(['battery_id',
                       dcc.Dropdown(id = 'id-dropdown',
                                    value = '1',
                                    options = [{'label': b, 'value': b} for b in df['battery_id'].unique()])
                        ]),
                    ], width = 12)
                    ]), #close row 2
     #------------------------------------------------------------------------------------------------------------------------
                      #Row 3: time series data plots
                    dbc.Row([
                        #========================================================================================
                        #Column 1 (left)
                        dbc.Col([
                       dcc.Graph(id= 'ts-graph'),
                       dcc.Dropdown(id = 'resampling-dropdown',
                                    value = '1MS',
                                    options = option_resampling)
                        ], 
                        width = {'size':5, 'offset': 1}), #close column 1
                        #===========================================================================================
                        #Column 2 (right)
                        dbc.Col([
                            dcc.Graph(id = 'stat-boxplot')
                        ], 
                        width = {'size':5, 'offset': 1})
                        ]), #close Row 3
    #------------------------------------------------------------------------------------------------------------------------
                    #Row 4: sub section title and text
                    dbc.Row([
                        dbc.Col([
                      html.H2('Clustering analysis results', className = 'text-center mb-4'),
                      html.P("Some explanation here", className = 'text-center mb-4'),
                      ], width = 12)
                    ]), #close row 4
    #-------------------------------------------------------------------------------------------------------------------------
                    #Row 5: clustering plots
                    dbc.Row([
                        #=======================================================================================
                        #Column 1
                        dbc.Col([
                            
                            '''
                             Etienne
                            
                            Please add here the "dynamic" plot with the clusters being formed.
                            '''   
                        ], width = 5), # close column #1
                        #========================================================================================
                        #Column 2
                        dbc.Col([
                            # '''
                            # Elias
                            
                            # Please add here the heatmaps of your clustering.
                            # ''' 
                            dcc.Graph(id= 'clustering_heatmaps'),                       
                            dcc.Dropdown(id = 'feature-type-dropdown',
                                value = 'Temperature features',
                                options = option_feature_type)
                        ], width = 5) #close column 2
                    ]), #close Row 5
    #------------------------------------------------------------------------------------------------------------------------
                #Row 6: sub title and text
                      dbc.Row([
                        dbc.Col([
                      html.H2('Extracting user profiles', className = 'text-center mb-4'),
                      html.P("Some explanation here", className = 'text-center mb-4'),
                      ], width = 12)
                    ]), #close row 6
    #------------------------------------------------------------------------------------------------------------------------
                #Row 7: User profile plots
                    dbc.Row([
                        #=======================================================================================
                        #Column 1
                        dbc.Col([
                            
                            '''
                             Etienne
                            
                            Please add here the average current and voltage coming out of the clustering analysis.
                            '''   
                        ], width = 7), # close column #1
                        #========================================================================================
                        #Column 2
                        dbc.Col([
                                                        '''
                             Maria
                             
                             Score card?
                            '''  
                        ], width = 5) #close column 2
                    ]), #close Row 7
]) #close the dbc container


## Callbacks

In [162]:
#callback + function for time series plot (plot #1)
@app.callback(
    Output(component_id = 'ts-graph', component_property = 'figure'),
    [
    Input(component_id = 'id-dropdown', component_property = 'value'),
    Input(component_id = 'var-dropdown', component_property = 'value'),
    Input(component_id = 'resampling-dropdown', component_property = 'value'),
    ]
)

def update_figure(selected_id, selected_var, resampling):
    df_plot = df[df['battery_id'] == selected_id]
    df_plot = df_plot.resample(resampling).mean()
    line_fig = px.line(df_plot.reset_index(), x = 'timestamp', y = selected_var, title = 'Time series plot')
    line_fig.show()
    return line_fig

In [163]:
#callback + function for box plot (plot #2)
@app.callback(
    Output(component_id = 'stat-boxplot', component_property = 'figure'),
    [
    Input(component_id = 'id-dropdown', component_property = 'value'),
    Input(component_id = 'var-dropdown', component_property = 'value')
    ]
)

def update_boxplot(selected_id, selected_var):
    mean_per_freq = df[[selected_var,'battery_id']].groupby('battery_id').resample('1M')[selected_var].agg('mean')
    mean_per_freq = mean_per_freq.reset_index()
    mean_per_freq['timestamp'] = mean_per_freq['timestamp'].dt.date
    mean_per_freq = mean_per_freq.sort_values('timestamp')
    fig = px.box(mean_per_freq, x = 'timestamp', y = selected_var, title = 'Monthly box plot');
    return fig

In [164]:
# import matplotlib
# import matplotlib.pyplot as plt
# import seaborn as sns
# # import plotly.plotly as py

In [165]:
#callback + function for heatmap plot (plot #4)
@app.callback(
    Output(component_id = 'clustering_heatmaps', component_property = 'figure'),
    [
    Input(component_id = 'feature-type-dropdown', component_property = 'value'),
    ]
)

def update_boxplot(selected_feature_type):
    if selected_feature_type == "Temperature features":
        # fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,5),sharey=True, gridspec_kw={'width_ratios': [1, 20]})
        # plt.subplots_adjust(wspace=0.05)
        # g1 = sns.heatmap(df_temp_features_clustered.sort_values('cluster').iloc[:,-1:], annot=False, cmap='Set2', ax=ax1, cbar=False)
        # g2 = sns.heatmap(df_temp_features_clustered.sort_values('cluster').iloc[:,:-1], annot=False, cmap='coolwarm',cbar_kws={'label': 'Temperature'}, ax=ax2)
        # ax1.set_ylabel('Battery_id')
        # ax2.set_title('Clusters and Temperature Features')

        data = df_temp_features_clustered.sort_values('cluster').iloc[:,:-1]
        xlabels = []
        ylabels = []
        for entry in data.columns.values:
            xlabels.append(entry)
        for entry in data.index.values:
            ylabels.append(entry)
        fig = px.imshow(data, labels = dict(x="Features", y="Battery id"), 
            x = xlabels, y = ylabels)
        fig.update_xaxes(side="top")
        fig.show()
        
    # mean_per_freq = df[[selected_var,'battery_id']].groupby('battery_id').resample('1M')[selected_var].agg('mean')
    # mean_per_freq = mean_per_freq.reset_index()
    # mean_per_freq['timestamp'] = mean_per_freq['timestamp'].dt.date
    # mean_per_freq = mean_per_freq.sort_values('timestamp')
    # fig = px.box(mean_per_freq, x = 'timestamp', y = selected_var, title = 'Monthly box plot');
    # return fig
    return fig

In [166]:
data = df_temp_features_clustered.sort_values('cluster').iloc[:,:-1]

In [167]:
ylabels

[99,
 128,
 127,
 123,
 118,
 114,
 113,
 111,
 108,
 106,
 104,
 103,
 101,
 100,
 198,
 98,
 96,
 95,
 74,
 77,
 78,
 80,
 81,
 82,
 132,
 83,
 86,
 90,
 91,
 92,
 93,
 94,
 85,
 133,
 138,
 139,
 175,
 176,
 181,
 182,
 183,
 184,
 174,
 186,
 188,
 190,
 191,
 192,
 194,
 196,
 187,
 73,
 172,
 165,
 140,
 141,
 146,
 147,
 148,
 153,
 169,
 155,
 157,
 159,
 160,
 161,
 162,
 163,
 156,
 72,
 199,
 19,
 56,
 55,
 34,
 18,
 52,
 39,
 41,
 17,
 50,
 23,
 24,
 32,
 46,
 45,
 30,
 44,
 22,
 28,
 58,
 8,
 38,
 68,
 3,
 4,
 37,
 7,
 65,
 16,
 29,
 12,
 61,
 35,
 40,
 70,
 15,
 64,
 130,
 31,
 136,
 71,
 151,
 25,
 21,
 167,
 173,
 14,
 13,
 179,
 11,
 5,
 193,
 150,
 126,
 135,
 76,
 43,
 54,
 125,
 67,
 75,
 116,
 110,
 112,
 48,
 62,
 154,
 26,
 84,
 20,
 51,
 177,
 164,
 168,
 87,
 170,
 49,
 105,
 88,
 189,
 109,
 122,
 145,
 144,
 129,
 171,
 197,
 1,
 59,
 60,
 195,
 2,
 10,
 6,
 66,
 178,
 79,
 180,
 185,
 63,
 69,
 9,
 124,
 57,
 121,
 120,
 131,
 36,
 119,
 134,
 117,
 137,
 11

In [168]:
if __name__ == '__main__':
    app.run_server(debug = True, port = 1000)

Dash app running on http://127.0.0.1:1000/
